In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import Rangel
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
from shapely.geometry import LineString
from linref.events.geometry import LineStringM, linemerge_m
import shapely

l1 = shapely.from_wkt('LINESTRING (0 0, 1 0, 1 1, 2 1, 3 2)')
l2 = shapely.from_wkt('LINESTRING (3 2, 4 3, 5 3, 6 4, 7 5)')
l3 = shapely.from_wkt('LINESTRING (8 5, 8 6, 9 7, 10 8, 11 9)')
lm1 = LineStringM(l1, [1, 5, 20, 30, 40])
lm2 = LineStringM(l2, [40, 60, 70, 80, 90])
lm3 = LineStringM(l3, [100, 110, 120, 130, 140])

In [3]:
l12 = shapely.ops.linemerge([l1, l2])
l123 = shapely.ops.linemerge([l1, l2, l3])
l12.wkt, l123.wkt

('LINESTRING (0 0, 1 0, 1 1, 2 1, 3 2, 4 3, 5 3, 6 4, 7 5)',
 'MULTILINESTRING ((0 0, 1 0, 1 1, 2 1, 3 2, 4 3, 5 3, 6 4, 7 5), (8 5, 8 6, 9 7, 10 8, 11 9))')

In [7]:
linemerge_m([lm2, lm1], allow_multiple=True, squeeze=False)

[LINESTRING M (0.0 0.0 1, 1.0 0.0 5, 1.0 1.0 20, 2.0 1.0 30, 3.0 2.0 40, 4.0 3.0 60, 5.0 3.0 70, 6.0 4.0 80, 7.0 5.0 90) # linref compatibility approximation]

In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import Rangel
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
df_linear_1 = pd.DataFrame(index=[7, 6, 9, 4, 3, 2, 0, 1], data={
    'county': ['A', 'A', 'A', 'A', 'A' ,'B', 'B', 'B'],
    'route': ['Main St', 'Main St', 'Main St', '1st Ave', '1st Ave', '2nd Ave', '2nd Ave', 'Sycamore School Rd'],
    'beg': [0.0, 0.5, 1.1, 10.0, 12.0, 13.0, 13.1, 1.8],
    'end': [0.4, 1.1, 3.0, 12.0, 13.0, 13.1, 14.0, 3.0],
    'volume': [100, 200, 100, 400, 500, 150, 200, 800],
    'class': ['B', 'A', 'A', 'A', 'B', 'A', 'A', 'A'],
})
#df_linear_1 = pd.concat([df_linear_1] * 100, ignore_index=True)#.sort_values(['route', 'beg'])
df_linear_2 = pd.DataFrame({
    'route': ['Main St', 'Main St', '1st Ave', '2nd Ave', '2nd Ave', 'Sycamore School Rd'],
    'beg': [0.0, 1.0, 10.0, 13.0, 13.1, 2.2],
    'end': [1.0, 2.0, 12.0, 13.0, 14.0, 2.8],
    'street_type': ['A', 'B', 'A', 'A', 'B', 'A'],
})
#df_linear_2 = pd.concat([df_linear_2] * 100, ignore_index=True)#.sort_values(['route', 'beg'])

In [3]:
rng1 = Rangel(index=df_linear_1.index, groups=df_linear_1.route, begs=df_linear_1.beg, ends=df_linear_1.end, closed='left_mod')
rng2 = Rangel(index=df_linear_2.index, groups=df_linear_2.route, begs=df_linear_2.beg, ends=df_linear_2.end, closed='left_mod')
rng3 = Rangel(index=df_linear_1.index, groups=df_linear_1.route, locs=df_linear_1.beg)

In [4]:
from linref.ext.base import LRS
lrs = LRS(['county', 'route'], 'loc', 'beg', 'end', 'geom', closed='left_mod')
df = pd.concat([df_linear_1] * 10000, ignore_index=True)
df.lr.set_lrs(lrs)

In [5]:
df.lr._lrs

[LRS(keys_col=['county', 'route'], locs_col=loc, begs_col=beg, ends_col=end, geom_col=geom, closed=left_mod)]

In [6]:
df.lr.dissolve(retain=['class'])

,county,route,class,beg,end,dissolved_index
0,A,Main St,B,0.0,0.4,[0]
1,A,Main St,A,0.5,3.0,"[1, 2]"
2,A,1st Ave,A,10.0,12.0,[3]
3,A,1st Ave,B,12.0,13.0,[4]
4,B,2nd Ave,A,13.0,14.0,"[5, 6]"
...,...,...,...,...,...,...
59995,A,Main St,A,0.5,3.0,"[79993, 79994]"
59996,A,1st Ave,A,10.0,12.0,[79995]
59997,A,1st Ave,B,12.0,13.0,[79996]
59998,B,2nd Ave,A,13.0,14.0,"[79997, 79998]"


In [7]:
ev = df.lr.events
ev

0, group(('A', 'Main St')    ) [ 0.000,  0.400]
1, group(('A', 'Main St')    ) [ 0.500,  1.100)
2, group(('A', 'Main St')    ) [ 1.100,  3.000]
3, group(('A', '1st Ave')    ) [10.000, 12.000)
4, group(('A', '1st Ave')    ) [12.000, 13.000]
...79,990 records...
79995, group(('A', '1st Ave')    ) [10.000, 12.000)
79996, group(('A', '1st Ave')    ) [12.000, 13.000]
79997, group(('B', '2nd Ave')    ) [13.000, 13.100)
79998, group(('B', '2nd Ave')    ) [13.100, 14.000]
79999, group(('B', 'Sycamore S...) [ 1.800,  3.000]
Rangel(80,000 grouped, monotonic linear events, closed=left_mod)

In [9]:
ev.relate(ev).intersect()

(80000, 80000)